## Human or Bot Detection Using Bidirectional LSTM


In [4]:
import pandas as pd

In [16]:
df=pd.read_csv('preprocessed_filename (1).csv')

In [17]:
df.head()

,Texts,content,corpus
0,I buy this iPhone last 2 weeks.From my experi...,human,buy iphon last week experi iphon good perform ...
1,The impact of renewable energy sources on the ...,bot,impact renew energi sourc mine industri natur ...
2,You don't have anything to complaint when you...,human,anyth complaint get iphon big billion day sale...
3,Awesome experience nice product from apple.ca...,human,awesom experi nice product appl camera experi ...
4,Super fantastic Apple iPhone...I repent not t...,human,super fantast appl iphon repent earlier beast ...


In [18]:
###Drop Nan Values
df=df.dropna()


In [19]:
## Get the Independent Features

X=df.drop(['content', 'Texts'],axis=1)

In [10]:
## Get the Dependent features
y=df['content']

In [11]:
y.value_counts()

human    850
bot      680
Name: content, dtype: int64

In [12]:
X.shape

(1530, 1)

In [13]:
y.shape

(1530,)

In [14]:
import tensorflow as tf

In [15]:
tf.__version__

'2.12.0'

In [20]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [21]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [22]:
import nltk
import re


In [23]:
df['corpus']

0       buy iphon last week experi iphon good perform ...
1       impact renew energi sourc mine industri natur ...
2       anyth complaint get iphon big billion day sale...
3       awesom experi nice product appl camera experi ...
4       super fantast appl iphon repent earlier beast ...
                              ...                        
1525    qualiti quit good satisfiedbtw iphon phone inc...
1526    use artifici intellig market advertis present ...
1527    first iphon love perform camera user interfac ...
1528    impact digit technolog creativ innov constantl...
1529    impact digit technolog journal media constantl...
Name: corpus, Length: 1530, dtype: object

In [24]:
onehot_repr=[one_hot(words,voc_size)for words in df['corpus']] 
onehot_repr

[[71,
  1664,
  4741,
  3060,
  2678,
  1664,
  4501,
  2769,
  2595,
  326,
  2772,
  3879,
  3080,
  3382,
  3873,
  1832,
  326,
  3622,
  4875,
  1784,
  2678,
  4965,
  483,
  3425,
  2569,
  4121,
  2569,
  4581,
  1337,
  3218,
  73],
 [3760,
  490,
  3527,
  1054,
  2000,
  3005,
  4180,
  2771,
  441,
  1937,
  4849,
  454,
  2443,
  1812,
  3463,
  2467,
  1827],
 [3354, 1405, 1245, 1664, 394, 16, 31, 3509, 3248, 2523, 1664, 4439, 4456],
 [3853,
  2678,
  3934,
  4269,
  4581,
  1553,
  2678,
  3853,
  2046,
  326,
  669,
  3573,
  3879,
  4978,
  2046,
  2769,
  71,
  1664,
  4618,
  1854,
  797],
 [4978,
  584,
  4581,
  1664,
  65,
  65,
  3641,
  3874,
  2902,
  1433,
  1318,
  4978,
  1553,
  3503,
  4741,
  326,
  919,
  3187,
  3776,
  3698,
  1664,
  3048,
  2250,
  2868],
 [2737,
  1664,
  4287,
  3248,
  4357,
  1325,
  2071,
  3622,
  3409,
  4627,
  1664,
  578,
  1756,
  1664,
  1028,
  1664,
  1337,
  4623,
  3665],
 [4357,
  2585,
  1751,
  4269,
  1812,
  3535

In [25]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[3879 3080 3382 ... 1337 3218   73]
 [   0    0    0 ... 3463 2467 1827]
 [   0    0    0 ... 1664 4439 4456]
 ...
 [   0    0    0 ...  313 4947 2642]
 [   0    0    0 ... 1929 3479 4269]
 [   0    0    0 ... 1862 4269 2728]]


In [22]:
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
len(embedded_docs),y.shape

(1530, (1530,))

In [24]:
from sklearn.preprocessing import LabelEncoder

# Create an instance of LabelEncoder
le = LabelEncoder()

# Fit the LabelEncoder to the categorical data and transform it into numerical labels
y = le.fit_transform(y)

# Print the numerical labels
print(y)

[1 0 1 ... 1 0 0]


In [25]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [26]:
X_final.shape,y_final.shape

((1530, 20), (1530,))

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [28]:
X_train

array([[   0,    0,    0, ..., 1370, 3808, 3293],
       [   0,    0,    0, ..., 1682, 1824, 3203],
       [   0,    0,    0, ..., 1682, 1824, 3203],
       ...,
       [   0,    0,    0, ..., 4687, 3520, 4220],
       [   0,    0,    0, ...,  242, 4702, 1722],
       [   0,    0,    0, ..., 2761, 2150, 4629]])

In [29]:
y_train

array([0, 0, 0, ..., 0, 0, 0])

### Model Training

In [30]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,batch_size=64)

Epoch 1/100
17/17 [==============================] - 5s 100ms/step - loss: 0.6466 - accuracy: 0.6078 - val_loss: 0.5221 - val_accuracy: 0.6792
Epoch 2/100
17/17 [==============================] - 1s 55ms/step - loss: 0.2650 - accuracy: 0.8976 - val_loss: 0.1726 - val_accuracy: 0.9446
Epoch 3/100
17/17 [==============================] - 1s 56ms/step - loss: 0.0818 - accuracy: 0.9795 - val_loss: 0.1423 - val_accuracy: 0.9545
Epoch 4/100
17/17 [==============================] - 1s 55ms/step - loss: 0.0298 - accuracy: 0.9941 - val_loss: 0.0787 - val_accuracy: 0.9743
Epoch 5/100
17/17 [==============================] - 1s 58ms/step - loss: 0.0122 - accuracy: 0.9990 - val_loss: 0.0785 - val_accuracy: 0.9723
Epoch 6/100
17/17 [==============================] - 1s 55ms/step - loss: 0.0044 - accuracy: 0.9990 - val_loss: 0.0851 - val_accuracy: 0.9762
Epoch 7/100
17/17 [==============================] - 1s 64ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.0825 - val_accuracy: 0.9802
Epoch

17/17 [==============================] - 1s 56ms/step - loss: 2.4931e-05 - accuracy: 1.0000 - val_loss: 0.1412 - val_accuracy: 0.9782
Epoch 58/100
17/17 [==============================] - 1s 50ms/step - loss: 2.4563e-05 - accuracy: 1.0000 - val_loss: 0.1416 - val_accuracy: 0.9782
Epoch 59/100
17/17 [==============================] - 1s 58ms/step - loss: 2.3431e-05 - accuracy: 1.0000 - val_loss: 0.1402 - val_accuracy: 0.9782
Epoch 60/100
17/17 [==============================] - 1s 54ms/step - loss: 2.3555e-05 - accuracy: 1.0000 - val_loss: 0.1407 - val_accuracy: 0.9782
Epoch 61/100
17/17 [==============================] - 1s 52ms/step - loss: 2.0251e-05 - accuracy: 1.0000 - val_loss: 0.1419 - val_accuracy: 0.9782
Epoch 62/100
17/17 [==============================] - 1s 48ms/step - loss: 2.2449e-05 - accuracy: 1.0000 - val_loss: 0.1401 - val_accuracy: 0.9782
Epoch 63/100
17/17 [==============================] - 1s 47ms/step - loss: 2.0461e-05 - accuracy: 1.0000 - val_loss: 0.1415 - val_a

### Performance Metrics And Accuracy

In [31]:

y_pred1=model1.predict(X_test)

16/16 [==============================] - 1s 10ms/step


In [32]:
y_pred1

array([[9.9999017e-01],
       [6.7067333e-07],
       [9.9998987e-01],
       [6.4099999e-07],
       [6.3410153e-07],
       [6.9081614e-07],
       [1.0437666e-06],
       [9.9999171e-01],
       [9.9999762e-01],
       [7.8383096e-07],
       [1.0338854e-06],
       [9.9999005e-01],
       [1.4876831e-06],
       [6.7782548e-07],
       [6.5495175e-07],
       [6.8893780e-07],
       [9.9993938e-01],
       [1.4245968e-06],
       [7.1011999e-07],
       [6.1242633e-07],
       [9.9999005e-01],
       [9.9998915e-01],
       [9.2685541e-06],
       [7.9707632e-07],
       [4.5930591e-04],
       [9.9999040e-01],
       [9.3907067e-07],
       [2.6755910e-05],
       [2.7828262e-04],
       [6.2316610e-07],
       [1.0286159e-06],
       [8.1542612e-07],
       [9.9998719e-01],
       [6.6642326e-07],
       [7.5149177e-07],
       [8.7233519e-07],
       [7.7901245e-07],
       [9.9999195e-01],
       [9.9999529e-01],
       [9.9999255e-01],
       [9.9999285e-01],
       [9.999918

In [33]:
def process(x):
    if(x>=0.5):
        return 1
    return 0

In [34]:
import numpy as np
y_pred1 = np.vectorize(process)(y_pred1)

In [35]:
len(X_test)

505

In [36]:
y_pred1

array([[1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
    

In [37]:
from sklearn.metrics import confusion_matrix

In [38]:
confusion_matrix(y_test,y_pred1)

array([[215,   7],
       [  4, 279]], dtype=int64)

In [39]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.9782178217821782

In [40]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.98      0.97      0.98       222
           1       0.98      0.99      0.98       283

    accuracy                           0.98       505
   macro avg       0.98      0.98      0.98       505
weighted avg       0.98      0.98      0.98       505



In [41]:
import pickle

# save the model to a file using pickle
with open('model.pkl', 'wb') as f:
    pickle.dump(model1, f)


In [42]:

# load the model from the file
with open('model.pkl', 'rb') as f:
    clf = pickle.load(f)

# make predictions on new data

y_pred = clf.predict(X_test)


16/16 [==============================] - 1s 7ms/step


In [43]:
y_pred

array([[9.9999017e-01],
       [6.7067333e-07],
       [9.9998987e-01],
       [6.4099999e-07],
       [6.3410153e-07],
       [6.9081614e-07],
       [1.0437666e-06],
       [9.9999171e-01],
       [9.9999762e-01],
       [7.8383096e-07],
       [1.0338854e-06],
       [9.9999005e-01],
       [1.4876831e-06],
       [6.7782548e-07],
       [6.5495175e-07],
       [6.8893780e-07],
       [9.9993938e-01],
       [1.4245968e-06],
       [7.1011999e-07],
       [6.1242633e-07],
       [9.9999005e-01],
       [9.9998915e-01],
       [9.2685541e-06],
       [7.9707632e-07],
       [4.5930591e-04],
       [9.9999040e-01],
       [9.3907067e-07],
       [2.6755910e-05],
       [2.7828262e-04],
       [6.2316610e-07],
       [1.0286159e-06],
       [8.1542612e-07],
       [9.9998719e-01],
       [6.6642326e-07],
       [7.5149177e-07],
       [8.7233519e-07],
       [7.7901245e-07],
       [9.9999195e-01],
       [9.9999529e-01],
       [9.9999255e-01],
       [9.9999285e-01],
       [9.999918

In [44]:
def process1(x):
    if(x>=0.5):
        return "human"
    return "bot"

In [45]:
import numpy as np
y_pred1 = np.vectorize(process1)(y_pred)

In [46]:
y_pred1[0:5]

array([['human'],
       ['bot'],
       ['human'],
       ['bot'],
       ['bot']], dtype='<U5')

In [47]:
X_test[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0, 1579, 4829, 3455, 4940])

In [48]:
y_test[0:5]

array([1, 0, 1, 0, 0])

In [49]:
corpus ='product good use'

In [50]:
onehot_repr=[one_hot(corpus,voc_size)] 
onehot_repr

[[3961, 3808, 2358]]

In [51]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0 3961 3808 2358]]


In [52]:
X_final=np.array(embedded_docs)

In [53]:
y_pred = clf.predict(X_final)

1/1 [==============================] - 0s 23ms/step


In [54]:
y_pred

array([[0.9999907]], dtype=float32)

In [55]:
process1(y_pred)

'human'

In [52]:
import pandas as pd

# Load the first CSV file
df1 = pd.read_csv('texts_data.csv')

# Load the second CSV file
df2 = pd.read_csv('myexcel.csv')

# Merge the two dataframes based on the 'Texts' and 'content' columns
merged_df = pd.merge(df1, df2, on=['Texts', 'content'])

# Write the merged dataframe to a new CSV file
merged_df.to_csv('merged_file.csv', index=False)
